In [50]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [51]:
df = pd.read_csv(r"C:\Users\sakti\OneDrive\Desktop\MindTrack\test.csv")

In [52]:
df.head()

,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0


In [53]:

# 🔁 Map label numbers to emotion strings
label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}
df['label'] = df['label'].map(label_map)





In [54]:
label_encoder = LabelEncoder()
y_labels = df['label']
y_encoded = label_encoder.fit_transform(y_labels)
y = to_categorical(y_encoded)

In [55]:

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, maxlen=100)


In [57]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)



In [58]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=100))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

c:\Users\sakti\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 0.3041 - loss: 1.7161 - val_accuracy: 0.3550 - val_loss: 1.5376
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.3175 - loss: 1.6265 - val_accuracy: 0.3725 - val_loss: 1.5300
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 102ms/step - accuracy: 0.3876 - loss: 1.5082 - val_accuracy: 0.4425 - val_loss: 1.4539
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.5667 - loss: 1.1468 - val_accuracy: 0.5025 - val_loss: 1.3586
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.7401 - loss: 0.7286 - val_accuracy: 0.5550 - val_loss: 1.2994
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.8546 - loss: 0.4236 - val_accuracy: 0.6000 - val_loss: 1.2706
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - accuracy: 0.8871 - loss: 0.3445 - val_accuracy: 0.5850 - val_loss: 1.4799
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.9424 - loss: 0.1917 - val_accuracy: 0.6225 

In [59]:
model.save('text_emotion_model.h5')
print("✅ Model training complete and saved.")

✅ Model training complete and saved.


In [60]:
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [61]:
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

In [62]:
model = load_model('text_emotion_model.h5')

In [64]:
classes = label_encoder.classes_